In [68]:
import pandas as pd
import numpy as np
import warnings
import re
from datetime import datetime
from collections import OrderedDict
warnings.filterwarnings('default')

In [12]:
# подгрузка всех файлов
import os
dr = 'exp'
file_names = os.listdir(dr)
file_names.remove('prepared')
# файлы в список
df_dicts = {}
for name in file_names:
    df_dicts[name]= pd.read_excel(dr+'\\'+name, None)
#     print(name)

In [13]:
# список всех таблиц из всех файлов
sheetname_list = []
for name, dict_ in zip(df_dicts, df_dicts.values()):    
    tmp = dict_.keys()
    tmp = list(tmp)
    tmp = tmp[2:-3]
    sheetname_list.append(tmp)
len(sheetname_list)

27

In [14]:
# dict_sheet_count - словарь всех вкладок(листов) из таблиц без дубликатов и их частоты
# uniq_sheetname - множество всех вкладок(листов) из таблиц без дубликатов (не используется)

dict_sheet_count = {}
uniq_sheetname = set()
for st in sheetname_list:
    for ls in st:
        uniq_sheetname.add(ls)


uniq_sheetname = list(uniq_sheetname)
uniq_sheetname.sort()
print(len(uniq_sheetname))

# добавление частот в словарь
for st in sheetname_list:
    for ls in st:
        counts = dict_sheet_count.get(ls,0)
        dict_sheet_count[ls] = counts + 1
        
        
# df - для вывода словаря в excel            
df = pd.DataFrame.from_dict(data = dict_sheet_count, orient='index')
# df.to_excel('dict_sheet_count.xlsx')

91


In [6]:
# найти файл по вкладке и вывести размер вкладки
def findfile(sheetname):
    row = 0
    col = 0
    for name, df_l in df_dicts:
        try:
            col = df_l[sheetname].shape[1]
            df_l[sheetname]
            print(name)
        except:
            continue
        try:
            row += df_l[sheetname].shape[0] 
            if (abs(col - col)<1):
                   continue
            else:
                col += abs(df_l[sheetname].shape[1] - col)
        except:
            continue
    print('rows = %d, col = %d' %(row, col))


In [155]:
ls_mb_convert = [
    'Предел текучести (1141)',
    'Предел текучести (1027)',
    'Предел текучести, Н/мм2 или МПа',
    'Предел текучести , кгс/мм2',
    'Предел текучести, кгс/мм2',
    
    
    'Врем. сопротивление,  Н/мм2 или МПа',
    'Врем. сопротивление (1140)',
    'Врем. сопротивление (1042)',
    'Врем. сопротивление , кгс/мм2',
    'Врем. сопротивление, кгс/мм2'
    
]

ls_mb_convert_ys = [
    'Предел текучести (1141)',
    'Предел текучести (1027)',
    'Предел текучести, Н/мм2 или МПа',
    'Предел текучести , кгс/мм2',
    'Предел текучести, кгс/мм2',
]

ls_mb_convert_tr = [
    'Врем. сопротивление,  Н/мм2 или МПа',
    'Врем. сопротивление (1140)',
    'Врем. сопротивление (1042)',
    'Врем. сопротивление , кгс/мм2',
    'Врем. сопротивление, кгс/мм2'
]

In [156]:
def change_col_name(df):
    for col in ls_mb_convert_tr:
        if col in df:
            df['Врем. сопротивление'] = df[col]
            del df[col]
    for col in ls_mb_convert_ys:
        if col in df:
            df['Предел текучести'] = df[col]
            del df[col]
    return df

In [8]:
ls_sheets = [
    u'Растяжение, кгс мм2.',
    u'Растяжение, н мм2.',
]

In [149]:
scretching = pd.DataFrame()
for filename in file_names:
    tmp = df_dicts[filename][u'Растяжение, кгс мм2.']
    tmp = change_col_name(tmp)
    scretching= pd.concat([tmp, scretching])

In [157]:
scretching2  = pd.DataFrame()
for filename in file_names:
    tmp = df_dicts[filename][u'Растяжение, н мм2.']
    tmp = change_col_name(tmp)
    scretching2= pd.concat([tmp, scretching2])

In [150]:
scr_df = scretching[[
    '№ плавки',
    '№ партии',
    'Дата форм. цехом',
    'Марка стали',
    'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести' ]]

In [158]:
scr_df2 = scretching2[[
    '№ плавки',
    '№ партии',
    'Дата форм. цехом',
    'Марка стали',
    'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести' ]]

In [42]:
scr_df= scr_df[scr_df['Тип предела текучести (1186)']==0.2]
# .describe()

In [159]:
scr_df2= scr_df2[scr_df2['Тип предела текучести (1186)']==0.2]
# .describe()

In [160]:
scr_df2.describe()

,Тип предела текучести (1186),Врем. сопротивление,Предел текучести
count,6045.0,5971.000000,5973.000000
mean,0.2,769.828303,660.176511
std,0.0,125.870970,135.497681
min,0.2,46.500000,26.000000
25%,0.2,728.500000,621.000000
50%,0.2,775.000000,670.000000
75%,0.2,816.000000,721.000000
max,0.2,1208.000000,1126.000000


In [143]:
scr_df.describe()

In [45]:
scr_df['Врем. сопротивление'] = scr_df['Врем. сопротивление'].apply(lambda x: x/9.8 if x>200 else x)
scr_df['Предел текучести'] = scr_df['Предел текучести'].apply(lambda x: x/9.8 if x>200 else x)

In [162]:
scr_df2['Врем. сопротивление'] = scr_df2['Врем. сопротивление'].apply(lambda x: x/9.8 if x>200 else x)
scr_df2['Предел текучести'] = scr_df2['Предел текучести'].apply(lambda x: x/9.8 if x>200 else x)

In [163]:
scr_df2.describe()

,Тип предела текучести (1186),Врем. сопротивление,Предел текучести
count,6045.0,5971.000000,5973.000000
mean,0.2,78.660608,67.447861
std,0.0,12.527520,13.624434
min,0.2,46.122449,26.000000
25%,0.2,74.336735,63.418367
50%,0.2,79.081633,68.367347
75%,0.2,83.316327,73.571429
max,0.2,123.265306,114.897959


In [164]:
scr_df = pd.concat([scr_df, scr_df2])

In [50]:
scr_df.to_excel('table_stretching_13april.xlsx')

In [165]:
scr_df['№ партии']=scr_df['№ партии'].apply(lambda x: str(x))

In [166]:
scr_df.shape

(27247, 7)

In [167]:
scr_df.drop_duplicates(inplace = True)
scr_df.shape

(26675, 7)

In [168]:
scr_df['Дата форм. цехом'] = scr_df['Дата форм. цехом'].apply(lambda x: datetime.strptime(x, "%d.%m.%Y"))
# dt = datetime.strptime("01.02.2016", "%d.%m.%Y")
# dt

In [169]:
scr_df.reset_index(inplace=True,drop=True)

In [170]:
undupl = scr_df[~scr_df[['№ плавки', '№ партии']].duplicated(keep = False)]

In [171]:
dupl_need_to_mean = scr_df[scr_df[['№ плавки', '№ партии']].duplicated(keep = False)]
# sort_values(['№ партии', 'Дата форм. цехом'])

In [172]:
dupl_need_to_mean.shape, undupl.shape, dupl_need_to_mean.shape[0]+undupl.shape[0]

((16990, 7), (9685, 7), 26675)

In [128]:
# d = {1:['11', '12'], 2:['21', '22']}
# ttt = pd.DataFrame(d)
# ttt
ttt[1].iloc[0]

'11'

In [173]:
def mean_func(x):
    if min(x['Дата форм. цехом'])!=max(x['Дата форм. цехом']):
        x = x[x['Дата форм. цехом']==max(x['Дата форм. цехом'])]
        x['Врем. сопротивление'] = x['Врем. сопротивление'].mean()
        x['Предел текучести'] = x['Предел текучести'].mean()
#         x['Марка стали'] = x['Марка стали'].iloc[0]
    else:
        x['Врем. сопротивление'] = x['Врем. сопротивление'].mean()
        x['Предел текучести'] = x['Предел текучести'].mean()
#         x['Марка стали'] = x['Марка стали'].iloc[0]
    return x.mean()

In [174]:
y = dupl_need_to_mean.groupby(['№ плавки', '№ партии'])['Дата форм. цехом',
    'Марка стали',
    'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести'].apply(mean_func)

In [176]:
# y.reset_index(inplace=True)
# y

In [105]:
dupl_need_to_mean.sort_values(['№ партии', 'Дата форм. цехом'])

In [177]:
# y[y[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values(['№ партии'])

,№ плавки,№ партии,Тип предела текучести (1186),Врем. сопротивление,Предел текучести


In [178]:
tmp = undupl[['№ плавки', '№ партии', 'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести']]
y = pd.concat([y, tmp])

In [179]:
y.shape

(16392, 5)

In [180]:
y.to_excel('table_stretching_13april.xlsx')